## 0. Imports

In [1]:
# restart your notebook if prompted on Colab
try:
    import verta
except ImportError:
    !pip install verta

In [2]:
H# Ensure credentials are set up, if not, use below
import os
# os.environ['VERTA_EMAIL'] = 
# os.environ['VERTA_DEV_KEY'] = 
os.environ['VERTA_HOST'] = 'allstate.dev.verta.ai'

from verta import Client

client = Client(os.environ['VERTA_HOST'])

## 1. Register a model

In [3]:
registered_model = client.get_or_create_registered_model(
    name="census", labels=["research-purpose", "team-a"])

In [4]:
from verta.registry import VertaModelBase

class MyModel(VertaModelBase):
    def __init__(self, artifacts):
        self.weights = json.load(open(artifacts["weights"]))
    
    def predict(self, input):
        res = []
        for row in input:
            res.append(row[0] * self.weights[0] + row[1] * self.weights[1])
        return res

In [5]:
from verta.environment import Python

model_version = registered_model.create_standard_model(
    model_cls=MyModel,
    artifacts = {"weights" : [5, 6]},
    environment=Python(requirements=["json"]),
    name="v0",
    labels=["prototype"],
)

## 2. Now you can use the registered model for deployment, monitoring, etc.